In [ ]:
from bokeh.plotting import *
from bokeh.models import HoverTool, ColumnDataSource

In [ ]:
%matplotlib inline

In [ ]:
output_notebook()

In [ ]:
import pandas as pd
import numpy as np
from ggplot import *
from proscatter.parse import parse_fasta

In [ ]:
def compute_prob_vector(ps_file, prob_paired=True):
    '''
    Given a text file derived from the RNAfold output of the form
    
    i    j    sqrt(prob)   ubox
    
    computes a vector (dict) of probabilities for every nucleotide
    to be in paired (or unpaired) state.
    '''
    prob_vector = {}
    with open(ps_file) as fi:
        for line in fi.readlines():
            line = line.strip()
            posi, posj, sqrt_prob, box = line.split()
            i,j = int(posi), int(posj)
            curr_i = prob_vector.get(i, 0.)
            curr_j = prob_vector.get(j, 0.)
            prob_vector.update({
                i: curr_i + float(sqrt_prob)**2,
                j: curr_j + float(sqrt_prob)**2,
            })
    if not prob_paired:
        prob_vector = dict([(pos, 1-p) for pos,p in prob_vector.items()])
    return prob_vector

In [ ]:
def trange_df(base_name, trange=range(35,43), abs_value=True):
    '''
    Same as `compute_diff_df` but builds dataframe in a long format
    suitable for ggplot faceting.
    '''
    T0 = trange[0]
    prob0 = pd.Series(list(compute_prob_vector('%s_%d.txt' % (base_name, T0)).values()))
    chunks = []
    for temp in trange[1:]:
        df = pd.DataFrame()
        prob_vector = compute_prob_vector('%s_%d.txt' % (base_name,temp))
        df['pos'] = prob_vector.keys()
        if abs_value:
            df['Diff'] = abs(pd.Series(list(prob_vector.values())) - prob0)
        else:
            df['Diff'] = pd.Series(list(prob_vector.values())) - prob0
        df['Temp'] = temp
        chunks.append(df)
    return pd.concat(chunks)

In [ ]:
def compute_diff_df(base_name, trange=range(35,43), abs_value=True):
    '''
    Given the base_name for tab-delimited files containing base
    pairing probabilities calculated by RNAfold computes a
    dataframe containing probability difference vectors for each
    temperature value in the range relative to the lowest T in the
    range.
    '''
    T0 = trange[0]
    prob = compute_prob_vector('%s_%d.txt' % (base_name, T0))
    df = pd.DataFrame(prob.items(), columns=['Position', 'Prob_%d' % T0])
    for temp in trange[1:]:
        prob = compute_prob_vector('%s_%d.txt' % (base_name, temp))
        prob_key = 'Prob_%d' % temp
        df[prob_key] = pd.Series(prob.values())
        if abs_value:
            df['Diff_%d' % temp] = abs(df[prob_key] - df['Prob_%d' % T0])
        else:
            df['Diff_%d' % temp] = df[prob_key] - df['Prob_%d' % T0]
    return df

In [ ]:
df = trange_df('../data/ROSE1', range(25,46))

In [ ]:
df

In [ ]:
from bokeh.plotting import *
from bokeh.models import HoverTool, ColumnDataSource

In [ ]:
fasta = parse_fasta('../data/rose.fa')
for rec in fasta:
    id, seq = rec
    
#df['seq'] = 12 * [base for base in seq]
df['seq'] = [seq[x-1] for x in df['pos']]

In [ ]:
def heatmap_T_range(df, temps, title=None):
    from bokeh.models import LinearAxis, SingleIntervalTicker
    data = df[['pos', 'seq', 'Temp', 'Diff']]
    row_range = [str(x) for x in sorted(temps)]
    col_range = [str(x) for x in set(df['pos'])]
    source = ColumnDataSource(
        data=dict(
            row=[str(x) for x in data['Temp']],
            col=[str(x) for x in data['pos']],
            base=data['seq'],
            alphas=data['Diff'],
        )
    )

    p = figure(title=title,
               tools='resize,save,box_zoom,reset,pan',
               x_range=col_range, 
               y_range=row_range,
               x_axis_type=None,
               plot_width=800)

    p.rect('col', 'row', 0.9, 0.9, source=source, alpha='alphas', color='#FF7F00', line_color=None)
    p.grid.grid_line_color = None
    ticker = SingleIntervalTicker(interval=100, num_minor_ticks=10)
    xaxis = LinearAxis(ticker=ticker)
    p.add_layout(xaxis, 'below')
    show(p)

In [ ]:
temps = set(df['Temp'])
heatmap_T_range(df, temps, title='Human HSR1')

In [ ]:
df_dHSR = trange_df('data/dHSR1', range(25,38))

In [ ]:
fasta = parse_fasta('data/dHSR1.fa')
for rec in fasta:
    id, seq = rec
    
df_dHSR['seq'] = [seq[x-1] for x in df_dHSR['pos']]

In [ ]:
temps = set(df_dHSR['Temp'])
heatmap_T_range(df_dHSR, temps, title='Drosophila HSR-1')

In [ ]:
df_ROSE = trange_df('test/ROSE1', range(25,46))
fasta = parse_fasta('test/rose.fa')
for rec in fasta:
    id, seq = rec
    
df_ROSE['seq'] = [seq[x-1] for x in df_ROSE['pos']]

In [ ]:
temps = set(df_ROSE['Temp'])
heatmap_T_range(df_ROSE, temps, title='ROSE-1')

In [ ]:
def sequence_heatmap(df, temp, num_cols=60):
    from bokeh.models import LinearAxis, SingleIntervalTicker
    data = df[df['Temp'] == temp]
    num_rows = data.shape[0] / num_cols
    if data.shape[0] % num_cols:
        num_rows += 1

    rows = [(x / num_cols) + 1 for x in range(data.shape[0])]
    cols = [(x - x / num_cols * num_cols) + 1 for x in range(data.shape[0])]
    

    row_range = [str(x) for x in reversed(range(1,num_rows+1))]
    col_range = [str(x) for x in range(1, num_cols)]
    
    source = ColumnDataSource(
        data=dict(
            row=[str(x) for x in rows],
            col=[str(x) for x in cols],
            base=data['seq'],
            alphas=data['Diff'],
        )
    )
    
    p = figure(title='Human HSR1 prob diff', tools='resize,hover,save',
           x_range=col_range, y_range=row_range, x_axis_type=None, plot_width=800)

    p.rect('col', 'row', 0.9, 0.9, source=source, alpha='alphas', color='#FF7F00', line_color=None)
    text_props = {
        'source': source,
        'angle': 0,
        'color': 'black',
        'text_color': '#586e75',
        'text_align': 'center',
        'text_baseline': 'middle',
    }

    p.text(x=dict(field='col', units='data'),
           y=dict(field='row', units='data'),
           text=dict(field='base', units='data'),
           text_font_size='10pt', text_font_style='bold', **text_props)

    p.grid.grid_line_color = None
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
        ('base', '@base'),
        ('shape', '@alphas')
    ]
    ticker = SingleIntervalTicker(interval=10, num_minor_ticks=10)
    xaxis = LinearAxis(ticker=ticker)
    p.add_layout(xaxis, 'below')
    
    show(p)

In [ ]:
sequence_heatmap(df, temp=37)

In [ ]:
sequence_heatmap(df_ROSE, temp=37, num_cols=30)

In [ ]:
ggplot(df, aes(xmin='pos', xmax='pos+1', ymin=0, ymax='Diff')) + geom_rect() + facet_wrap('Temp')

In [ ]:
from pandas.stats.moments import rolling_mean

def compute_rolling_mean(data, window=10):
    df = data.copy()
    df['rmean'] = 0
    for temp in set(df['Temp']):
        df.loc[df['Temp'] == temp, 'rmean'] = rolling_mean(df[df['Temp'] == temp]['Diff'], window=window)
    return df

In [ ]:
df = trange_df('data/hHSR', range(32,48))
ggplot(compute_rolling_mean(df, window=15), aes(x='pos', y='rmean')) \
        + geom_line() \
        + scale_y_continuous(limits=(0,0.25)) \
        + facet_wrap('Temp')

In [ ]:
df = trange_df('data/dHSR1', range(25,37))
ggplot(compute_rolling_mean(df, window=12), aes(x='pos', y='rmean')) \
        + geom_line() \
        + facet_wrap('Temp')

In [ ]:
from IPython.html.widgets import interact

def plot_T(temp):
    p = ggplot(df[df['Temp'] == temp], aes(xmin='pos', xmax='pos+1', ymin=0, ymax='Diff')) + geom_rect()
    print p
    
interact(plot_T, temp=(36,47))

In [ ]:
trange = range(35,48)
df_hHSR = compute_diff_df('data/hHSR', trange)

In [ ]:
dfd = trange_df('data/dHSR1', range(25,38))
ggplot(dfd, aes(xmin='pos', xmax='pos+1', ymin=0, ymax='Diff')) + geom_rect() + facet_wrap('Temp')

In [ ]:
def get_sig_positions(df, trange=range(37,43), num_sigma=6):
    '''
    Given the dataframe of probability differences for a T range
    and the level of significannce in sigmas returns positions in the
    dataframe where the probability difference at the highest T
    exceeds the sigma threshold.
    '''
    colnames = ['Diff_%d' % temp for temp in trange[1:]]
    diff_cols = [df[colname] for colname in colnames]
    all_diff = pd.concat(diff_cols)
    mean = all_diff.mean()
    sigma = all_diff.std()
    threshold = num_sigma * sigma
    print 'Mean:\t%f\nSigma:\t%f\nThreshold:\t%f\n' % (mean, sigma, threshold)
    return df[abs(df['Diff_%d' % trange[-1]] - mean) > threshold].sort(['Position'])

get_sig_positions(df_hHSR, trange=trange, num_sigma=3)

In [ ]:
def plot_RMSD(df, trange=range(37,43)):
    df_sum = pd.DataFrame()
    df_sum['Temp'] = trange[1:]
    mean = df['Diff'].mean()
    df_sum['RMSD'] = [np.sqrt(((df[df['Temp'] == T]['Diff']-mean)**2).sum()) for T in trange[1:]]
    #df_sum['RMSD'] = [df[df['Temp'] == T]['Diff'].sum() for T in trange[1:]]
    p = ggplot(df_sum, aes(x='Temp', y='RMSD')) + geom_line()
    print p

In [ ]:
plot_RMSD(df, trange=range(32,47))

In [ ]:
plot_RMSD(df, trange=range(34,42))

In [ ]:
trange1 = range(25,38)
df_dHSR1 = compute_diff_df('data/dHSR1', trange1)
get_sig_positions(df_dHSR1, trange1)

In [ ]:
df_dHSR1 = trange_df('data/dHSR1', range(26,37))
plot_RMSD(df_dHSR1, trange=range(26,37))

In [ ]:
df_435 = compute_diff_df('data/hHSR435', trange)
get_sig_positions(df_435, trange=trange)